In [1]:
from pathlib import Path

from itertools import product

In [2]:
data_path = Path.home() / 'workstation' / 'dev' / 'Advent-of-Code-2020' / 'data' / 'day19_input.txt'

In [3]:
data_path.exists()

True

In [4]:
with open(data_path, 'r') as reader:
    expr_input = reader.read().strip()

In [5]:
rule_expressions, recd_messages = expr_input.split('\n\n')

In [6]:
recd_message_list = recd_messages.split('\n')

In [7]:
rules_list = rule_expressions.strip('\n').split('\n')

In [8]:
def create_rules_dict(rules_list):
    rules_dict = {}
    for rule in rules_list:
        dict_key_str, rule_str = rule.split(': ')
        dict_key = int(dict_key_str)
        rules_dict[dict_key] = rule_str.strip('\"')
    return rules_dict

def parse_rule_string(rule_string):
    split_string = rule_string.split(' | ')
    return [
        (x if x.isalpha() else list(map(int, x.split(' ')))) 
        for x in split_string
    ]

def is_string(entry):
    return isinstance(entry, str)

def get_rule(rules_dict, index):
    return rules_dict.get(index)

#### Part 1

In [9]:
rules_dict = create_rules_dict(rules_list)

In [10]:
rule_string = get_rule(rules_dict, 0)

In [11]:
def generate_strings(rule_string):
    rule_list = parse_rule_string(rule_string)
    if is_string(rule_list[0]):
        yield rule_list[0]
    else:
        for rules in rule_list:
            get_strings = map(lambda x: generate_strings(rules_dict.get(x)), rules)
            
            for string_combination in product(*get_strings):
                yield ''.join(string_combination)

In [12]:
boolean_list = (message in generate_strings(rule_string) for message in recd_message_list)

In [13]:
valid_messages = list(generate_strings(rule_string))

In [14]:
%%time
sum((message in valid_messages for message in recd_message_list))

CPU times: user 9.68 s, sys: 19 ms, total: 9.7 s
Wall time: 9.71 s


210

#### Part 2

Main observation: Valid string will be of the form [42]\*M([42][31])\*N, where [42] refers to any valid string created from rule 42, and [31] refers to any valid string from rule 31

In [15]:
# Re-write rules to allow for recursion

rules_dict[8] = '42 | 42 8'
rules_dict[11] = '42 31 | 42 11 31'

In [16]:
max_recd_message_len = max(map(len, recd_message_list))

In [17]:
max_recd_message_len

96

In [18]:
len(recd_message_list)

482

In [19]:
get_strings_42 = list(generate_strings(rules_dict.get(42)))
get_strings_31 = list(generate_strings(rules_dict.get(31)))

In [20]:
next(map(len, get_strings_42))

8

In [21]:
next(map(len, get_strings_31))

8

In [22]:
def check_messages_for_validity(msg_list, msg_length):
    valid_messages = []
    
    for message in msg_list:
        l = len(message)
        i_max = l//msg_length
        valid_flag = 0
        break_point = i_max-1
        
        try:
            break_point = next(
                msg_length*i
                for i in range(0, i_max)
                if message[msg_length*i : msg_length*(i+1)] not in get_strings_42
            )
        except StopIteration:
            pass
            
        # Strings from rule 31 must be *strictly* shorter than/occur less often than 42
        if l//2 >= break_point:
            valid_flag = 1
        
        try:
            valid_flag = next(
                1
                for i in range(break_point//msg_length, i_max)
                if message[msg_length*i : msg_length*(i+1)] not in get_strings_31
            )
        except StopIteration:
            pass
            
        if valid_flag == 0:
            valid_messages.append(message)
            
    return valid_messages

In [23]:
valid_messages = check_messages_for_validity(recd_message_list, 8)

In [24]:
len(valid_messages)

422